In [2]:
import pandas as pd
import pickle
import os
import os.path as osp
import torch
from torchvision import models, transforms
from PIL import Image

import tensorflow as tf
import h5py
import numpy as np
from collections import Counter

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scipy.sparse as sp
import math
from torch.nn.parameter import Parameter

In [3]:
file_dir = "c:/Users/96446/Documents/GitHub/data/mmkg/fb15k_db15k"


In [4]:
csv_file_path = r'C:\Users\96446\Documents\GitHub\learning-pytorch/pytorch/FB15K_Updated.csv'

df = pd.read_csv(csv_file_path)
df['Entity Type'] = df['Entity Type'].str.lower().str.strip().str.replace('[^\w\s]', '', regex=True)

entity_type_counts = df['Entity Type'].value_counts()
filtered_entity_type_counts = entity_type_counts[entity_type_counts >= 0]

print(filtered_entity_type_counts)

Entity Type
human                                     4495
film                                      1659
association football club                  608
city                                       330
music genre                                328
                                          ... 
complication                                 1
constitutional republic                      1
stopmotion animated television program       1
performing arts genre                        1
extracurricular activity                     1
Name: count, Length: 1066, dtype: int64


In [5]:
temp = []
with open('ent_ids_1', 'r') as fr:
    for line in fr:
        temp.append(line.strip().split())
df1 = pd.DataFrame(temp, columns=['Index', 'Freebase ID'],index=None)   

df_merged = df1.merge(df, how='left', on='Freebase ID')
df_merged.drop(columns=['Index'], inplace=True)
df_merged.to_csv('merged_data.csv', index=False, encoding='utf-8-sig')


In [6]:
df = pd.read_csv('merged_data.csv')

In [7]:
df[df['Entity Type'].isna()]

,Freebase ID,Entity Name,Entity Type
30,/m/03q5t,NaN,NaN
79,/m/01z4y,NaN,NaN
101,/m/08mbj5d,NaN,NaN
215,/m/01rk91,NaN,NaN
267,/m/01q03,NaN,NaN
...,...,...,...
14934,/m/05cxb38,NaN,NaN
14936,/m/0kj0,NaN,NaN
14937,/m/0qb7t,NaN,NaN
14940,/m/02jnw,NaN,NaN


In [17]:
from sklearn.metrics import pairwise_distances

with open('joint_emb', 'rb') as f:
    # 加载数据
    joint_emb = pickle.load(f)
joint_emb = F.normalize(joint_emb).cpu().detach().numpy()

joint_emb_subset = joint_emb[:14951]

na_indices = df[df['Entity Type'].isna()].index
for index in na_indices:
    current_tensor = joint_emb_subset[index].reshape(1, -1)
    distances = pairwise_distances(current_tensor, joint_emb_subset)
    distances[0, index] = np.inf
    closest_index = np.argmin(distances)

    closest_entity_type = df.loc[closest_index, 'Entity Type']
    df.loc[index, 'Entity Type'] = closest_entity_type

In [18]:
df[df['Entity Type'].isna()]

,Freebase ID,Entity Name,Entity Type


In [21]:
from transformers import BertTokenizer, BertModel
import torch

# 加载预训练的BERT模型及其分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

df = pd.read_csv('C:/Users\96446\Documents\GitHub\data\mmkg\FBDB15K/norm\ill_ent_type.csv')
types = df['Entity Type'].values
all_embeddings = []
for i in range(0, len(types)):
    encoded_input = tokenizer(types[i], return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
    embeddings = output.last_hidden_state.mean(dim=1)
    all_embeddings.append(embeddings)
# Concatenate all batches
all_embeddings = torch.cat(all_embeddings, dim=0)
print(all_embeddings.shape)

torch.Size([14951, 768])


In [28]:
with open('ill_ent_type.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)

In [32]:
with open('ill_ent_type', 'rb') as f:
    type_emb = pickle.load(f)

In [40]:
ill_ent_ids = []

with open('ill_ent_ids', "r", encoding="utf-8") as fr:
    for line in fr:
        params = line.strip("\n").split("\t")
        ill_ent_ids.append(tuple([int(x) for x in params]))
        
extended_type_emb = torch.zeros(27793, type_emb.size(1))
extended_type_emb[:14951] = type_emb

for src_idx, target_idx in ill_ent_ids:
    # 确保target_idx在新创建的部分
    if target_idx >= 14951:
        extended_type_emb[target_idx] = extended_type_emb[src_idx]

torch.Size([14951, 768])

In [52]:
with open('ill_ent_types', 'wb') as f:
    pickle.dump(extended_type_emb, f)

In [29]:
ent_ids_1 = []
with open('ent_ids_1', 'r') as fr:
    for line in fr:
        ent_ids_1.append(line.strip().split())

In [31]:
ent_ids_2 = []
with open('ent_ids_2', 'r') as fr:
    for line in fr:
        ent_ids_2.append(line.strip().split())

UnicodeDecodeError: 'gbk' codec can't decode byte 0x93 in position 2387: illegal multibyte sequence

In [30]:
ent_ids_1

[['0', '/m/027rn'],
 ['1', '/m/06cx9'],
 ['2', '/m/017dcd'],
 ['3', '/m/06v8s0'],
 ['4', '/m/07s9rl0'],
 ['5', '/m/0170z3'],
 ['6', '/m/01sl1q'],
 ['7', '/m/044mz_'],
 ['8', '/m/0cnk2q'],
 ['9', '/m/02nzb8'],
 ['10', '/m/02_j1w'],
 ['11', '/m/01cwm1'],
 ['12', '/m/059ts'],
 ['13', '/m/03h_f4'],
 ['14', '/m/011yn5'],
 ['15', '/m/01pjr7'],
 ['16', '/m/04nrcg'],
 ['17', '/m/02sdk9v'],
 ['18', '/m/07nznf'],
 ['19', '/m/014lc_'],
 ['20', '/m/05cvgl'],
 ['21', '/m/04kxsb'],
 ['22', '/m/02qyp19'],
 ['23', '/m/02d413'],
 ['24', '/m/02vk52z'],
 ['25', '/m/01crd5'],
 ['26', '/m/0q9kd'],
 ['27', '/m/0184jc'],
 ['28', '/m/09w1n'],
 ['29', '/m/0sx8l'],
 ['30', '/m/03q5t'],
 ['31', '/m/07y_7'],
 ['32', '/m/0gqng'],
 ['33', '/m/073hkh'],
 ['34', '/m/0b76d_m'],
 ['35', '/m/029j_'],
 ['36', '/m/04ztj'],
 ['37', '/m/012v9y'],
 ['38', '/m/014_x2'],
 ['39', '/m/012ljv'],
 ['40', '/m/0ds35l9'],
 ['41', '/m/05r4w'],
 ['42', '/m/015qsq'],
 ['43', '/m/02bjrlw'],
 ['44', '/m/04bdxl'],
 ['45', '/m/02s2ft'],
 ['

In [14]:
pkl_file_path  = 'C:/Users\96446\Documents\GitHub\data\mmkg\pkls/FBDB15K_id_img_feature_dict.pkl'

with open(pkl_file_path, 'rb') as file:
    data = pickle.load(file)

# 检查数据的类型
print(type(data))

<class 'dict'>


In [15]:
data[0]

array([1.3076725, 1.2305019, 1.5317996, ..., 0.       , 2.080137 ,
       0.       ], dtype=float32)

In [16]:
image_index = 'FB15K/FB15K_ImageIndex.txt'
result = {}
with open(image_index, 'r') as fr:
    for img_id in fr:
        value = img_id.split()
        result[value[0]] = value[1]

vgg16_h5 = r'C:\Users\96446\Documents\GitHub\data\mmkg\vgg16\FB15K_ImageData.h5'

img_emb = {}

In [172]:
class MultiModalEncoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MultiModalEncoder, self).__init__()
        
        self.modal_fc = nn.Linear(input_dim, output_dim)

    def forward(self, modal_features):
        modal_emb = self.modal_fc(modal_features)
        return modal_emb

image_feature_dim = 4096 
image_dim = 100
model = MultiModalEncoder(image_feature_dim, image_dim)

for key, value in result.items():
    f = h5py.File(vgg16_h5, 'r')
    img_features = torch.from_numpy(np.array(f[value])).float()
    img_emb[key] = model(img_features).to('cuda')

In [173]:
result3 = {}

for key in img_emb.keys():
    
    matching_rows = df[df['Freebase ID'] == key]
    if not matching_rows.empty:
        entity_type = matching_rows['Entity Type'].iloc[0]
        result3[key] = entity_type
    else:
        print(f"No match found for Freebase ID: {key}")

print(result3)

No match found for Freebase ID: /m/0d1w9
No match found for Freebase ID: /m/03pty
No match found for Freebase ID: /m/03ss47
No match found for Freebase ID: /m/0bynt
No match found for Freebase ID: /m/03t22m
No match found for Freebase ID: /m/08q1tg
No match found for Freebase ID: /m/02g7sp
No match found for Freebase ID: /m/02k84w
No match found for Freebase ID: /m/01z27
No match found for Freebase ID: /m/01v1d8
No match found for Freebase ID: /m/07tnp
No match found for Freebase ID: /m/018rn4
No match found for Freebase ID: /m/01y64
No match found for Freebase ID: /m/04q7r
No match found for Freebase ID: /m/0bm02
No match found for Freebase ID: /m/04qbv
No match found for Freebase ID: /m/0bp7w
No match found for Freebase ID: /m/0dz96
No match found for Freebase ID: /m/01nkt
No match found for Freebase ID: /m/09s1f
No match found for Freebase ID: /m/0k4j
No match found for Freebase ID: /m/0979zs
No match found for Freebase ID: /m/0g64p
No match found for Freebase ID: /m/08mh3kd
No matc

In [174]:
result3['/m/01m4kpp']

'human'

In [175]:
def read_ent2id(file_paths):
    ent2id_dict = {}
    ids = []
    for file_path in file_paths:
        id_set = set()
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                parts = line.strip().split("\t")
                if len(parts) == 2:
                    ent2id_dict[parts[1]] = int(parts[0])
                    id_set.add(int(parts[0]))
        ids.append(id_set)
    return ent2id_dict, ids

def read_triples(file_paths):
    triples = []
    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                parts = line.strip().split("\t")
                triples.append((parts[0], parts[1], parts[2]))
    return triples

def get_adjr(ent_size, triples, norm=False):
    print('getting a sparse tensor r_adj...')
    M = {}
    for tri in triples:
        if tri[0] == tri[2]:
            continue
        if (tri[0], tri[2]) not in M:
            M[(tri[0], tri[2])] = 0
        M[(tri[0], tri[2])] += 1
    ind, val = [], []
    for (fir, sec) in M:
        ind.append((fir, sec))
        ind.append((sec, fir))
        val.append(M[(fir, sec)])
        val.append(M[(fir, sec)])


    for i in range(ent_size):
        ind.append((i, i))
        val.append(1)

    if norm:
        ind = np.array(ind, dtype=np.int32)
        val = np.array(val, dtype=np.float32)
        adj = sp.coo_matrix((val, (ind[:, 0], ind[:, 1])), shape=(ent_size, ent_size), dtype=np.float32)
        return sparse_mx_to_torch_sparse_tensor(normalize_adj(adj))
    else:
        M = torch.sparse_coo_tensor(torch.LongTensor(ind).t(), torch.FloatTensor(val), torch.Size([ent_size, ent_size]))
        return M
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.FloatTensor(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse_coo_tensor(indices, values, size=shape)

def load_img(e_num, path):
    img_dict = pickle.load(open(path, "rb"))
    imgs_np = np.array(list(img_dict.values()))
    mean = np.mean(imgs_np, axis=0)
    std = np.std(imgs_np, axis=0)
    img_embd = np.array([img_dict[i] if i in img_dict else np.random.normal(mean, std, mean.shape[0]) for i in range(e_num)])
    return img_embd

def load_relation(e, KG, topR=1000):
    rel_mat = np.zeros((e, topR), dtype=np.float32)
    rels = np.array(KG)[:, 1]
    top_rels = Counter(rels).most_common(topR)
    rel_index_dict = {r: i for i, (r, cnt) in enumerate(top_rels)}
    for tri in KG:
        h = int(tri[0])
        r = int(tri[1])
        o = int(tri[2])
        if r in rel_index_dict:
            rel_mat[h][rel_index_dict[r]] += 1.
            rel_mat[o][rel_index_dict[r]] += 1.
    return np.array(rel_mat), rel_index_dict

def load_attr(fns, e, ent2id, topA):
    cnt = {}
    for fn in fns:
        with open(fn, 'r', encoding='utf-8') as f:
            for line in f:
                th = line[:-1].split('\t')
                if th[0] not in ent2id:
                    continue
                for i in range(1, len(th)):
                    if th[i] not in cnt:
                        cnt[th[i]] = 1
                    else:
                        cnt[th[i]] += 1
    fre = [(k, cnt[k]) for k in sorted(cnt, key=cnt.get, reverse=True)]
    attr2id = {}
    topA = min(1000, len(fre))
    for i in range(topA):
        attr2id[fre[i][0]] = i
    attr = np.zeros((e, topA), dtype=np.float32)
    for fn in fns:
        with open(fn, 'r', encoding='utf-8') as f:
            for line in f:
                th = line[:-1].split('\t')
                if th[0] in ent2id:
                    for i in range(1, len(th)):
                        if th[i] in attr2id:
                            attr[ent2id[th[0]]][attr2id[th[i]]] = 1.0
    return attr, attr2id

lang = [1, 2]
triples = read_triples([f"{file_dir}/triples_{i}" for i in lang])
ent2id_dict, ids = read_ent2id([f"{file_dir}/ent_ids_{i}" for i in lang])

a1 = os.path.join(file_dir, 'training_attrs_1')
a2 = os.path.join(file_dir, 'training_attrs_2')

ENT_NUM = len(ent2id_dict)
input_idx = torch.LongTensor(np.arange(ENT_NUM)).to('cuda')
input_dim = 300
adj = get_adjr(ENT_NUM, triples, norm=True)
adj = adj.to('cuda')
entity_emb = nn.Embedding(ENT_NUM, input_dim).to('cuda')

getting a sparse tensor r_adj...


In [176]:
img_vec_path = os.path.join(file_dir, 'FB15K_DB15K_id_img_feature_dict.pkl')
img_features = load_img(ENT_NUM, img_vec_path)
img_features = F.normalize(torch.FloatTensor(img_features)).to('cuda')
image_feature_dim = 4096 
image_dim = 300

model = MultiModalEncoder(image_feature_dim, image_dim).to('cuda')
img_embdding = model(img_features)

In [178]:
rel_features, rel2id = load_relation(ENT_NUM, triples, 1000)
rel_features = torch.Tensor(rel_features).to('cuda')

rel_feature_dim = 1000 
rel_dim = 300

model = MultiModalEncoder(rel_feature_dim, rel_dim).to('cuda')
rel_emb = model(rel_features)

In [179]:
att_features, attr2id = load_attr([a1, a2], ENT_NUM, ent2id_dict, 1000)
att_features = torch.Tensor(att_features).to('cuda')
att_feature_dim = 341 
att_dim = 300

model = MultiModalEncoder(att_feature_dim, att_dim).to('cuda')
att_emb = model(att_features)

In [181]:
n_units = [300, 300, 300]
n_heads = [2,2]
dropout = 0
attn_dropout = 0
instance_normalization = False
diag = True

class MultiHeadGraphAttention(nn.Module):
    """
    Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
    https://github.com/Diego999/pyGAT/blob/master/layers.py
    """

    def __init__(self, n_head, f_in, f_out, attn_dropout, diag=True, init=None, bias=False):
        super(MultiHeadGraphAttention, self).__init__()
        self.n_head = n_head
        self.f_in = f_in
        self.f_out = f_out
        self.diag = diag
        if self.diag:
            self.w = Parameter(torch.Tensor(n_head, 1, f_out))
        else:
            self.w = Parameter(torch.Tensor(n_head, f_in, f_out))
        self.a_src_dst = Parameter(torch.Tensor(n_head, f_out * 2, 1))
        self.attn_dropout = attn_dropout
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.special_spmm = SpecialSpmm()
        if bias:
            self.bias = Parameter(torch.Tensor(f_out))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter('bias', None)
        if init is not None and diag:
            init(self.w)
            stdv = 1. / math.sqrt(self.a_src_dst.size(1))
            nn.init.uniform_(self.a_src_dst, -stdv, stdv)
        else:
            nn.init.xavier_uniform_(self.w)
            nn.init.xavier_uniform_(self.a_src_dst)

    def forward(self, input, adj):
        output = []
        for i in range(self.n_head):
            N = input.size()[0]
            edge = adj._indices()
            if self.diag:
                h = torch.mul(input, self.w[i])
            else:
                h = torch.mm(input, self.w[i])

            edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1)  # edge: 2*D x E
            edge_e = torch.exp(-self.leaky_relu(edge_h.mm(self.a_src_dst[i]).squeeze()))  # edge_e: 1 x E
            # e_rowsum: N x 1
            e_rowsum = self.special_spmm(edge, edge_e, torch.Size([N, N]), torch.ones(size=(N, 1)).cuda() if next(self.parameters()).is_cuda else torch.ones(size=(N, 1)))

            edge_e = F.dropout(edge_e, self.attn_dropout, training=self.training)   # edge_e: 1 x E

            h_prime = self.special_spmm(edge, edge_e, torch.Size([N, N]), h)
            h_prime = h_prime.div(e_rowsum)

            output.append(h_prime.unsqueeze(0))

        output = torch.cat(output, dim=0)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        if self.diag:
            return self.__class__.__name__ + ' (' + str(self.f_out) + ' -> ' + str(self.f_out) + ') * ' + str(self.n_head) + ' heads'
        else:
            return self.__class__.__name__ + ' (' + str(self.f_in) + ' -> ' + str(self.f_out) + ') * ' + str(self.n_head) + ' heads'
        
class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
        assert indices.requires_grad is False
        a = torch.sparse_coo_tensor(indices, values, shape)
        ctx.save_for_backward(a, b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.matmul(b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = a.t().matmul(grad_output)
        return None, grad_values, None, grad_b

        
class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)

class GAT(nn.Module):
    def __init__(self, n_units, n_heads, dropout, attn_dropout, instance_normalization, diag):
        super(GAT, self).__init__()
        self.num_layer = len(n_units) - 1
        self.dropout = dropout
        self.inst_norm = instance_normalization
        if self.inst_norm:
            self.norm = nn.InstanceNorm1d(n_units[0], momentum=0.0, affine=True)
        self.layer_stack = nn.ModuleList()
        self.diag = diag
        for i in range(self.num_layer):
            f_in = n_units[i] * n_heads[i - 1] if i else n_units[i]
            self.layer_stack.append(MultiHeadGraphAttention(n_heads[i], f_in, n_units[i + 1], attn_dropout, diag, nn.init.ones_, False))

    def forward(self, x, adj):
        if self.inst_norm:
            x = self.norm(x)
        for i, gat_layer in enumerate(self.layer_stack):
            if i + 1 < self.num_layer:
                x = F.dropout(x, self.dropout, training=self.training)
            x = gat_layer(x, adj)
            if self.diag:
                x = x.mean(dim=0)
            if i + 1 < self.num_layer:
                if self.diag:
                    x = F.elu(x)
                else:
                    x = F.elu(x.transpose(0, 1).contiguous().view(adj.size(0), -1))
        if not self.diag:
            x = x.mean(dim=0)

        return x


model = GAT(n_units=n_units, n_heads=n_heads, dropout=dropout, attn_dropout=attn_dropout, instance_normalization=instance_normalization, diag=diag)
model = model.to('cuda')
gph_emb = model(entity_emb(input_idx), adj)

In [182]:
print("Graph embedding shape:", gph_emb.shape)
print("Image embedding shape:", img_embdding.shape)
print("Relation embedding shape:", rel_emb.shape) 
print("Attribute embedding shape:", att_emb.shape) 

Graph embedding shape: torch.Size([27793, 300])
Image embedding shape: torch.Size([27793, 300])
Relation embedding shape: torch.Size([27793, 300])
Attribute embedding shape: torch.Size([27793, 300])


In [183]:
class MultiModalFusion(nn.Module):
    def __init__(self, modal_num, with_weight=1):
        super().__init__()
        self.modal_num = modal_num
        self.requires_grad = True if with_weight > 0 else False
        self.weight = nn.Parameter(torch.ones((self.modal_num, 1)),
                                   requires_grad=self.requires_grad)

    def forward(self, embs):
        weight_norm = F.softmax(self.weight, dim=0)
        embs = [weight_norm[idx] * F.normalize(embs[idx]) for idx in range(self.modal_num) if embs[idx] is not None]
        hidden_states = torch.stack(embs, dim=1)
        joint_emb = hidden_states.mean(dim=1)
        return joint_emb

In [184]:
with_weight=1
inner_view_num = 4
fusion = MultiModalFusion(modal_num=inner_view_num,
                                       with_weight=with_weight).to('cuda')
joint_emb = fusion([img_embdding, att_emb, rel_emb, gph_emb]).to('cuda')

In [186]:
joint_emb.shape

torch.Size([27793, 300])

In [207]:
inception = joint_emb[3468]
kg2_embeddings = joint_emb[14950:]

similarity_scores = F.cosine_similarity(inception.unsqueeze(0), kg2_embeddings, dim=1)
top10_indices = torch.topk(similarity_scores, 10)

top10_global_indices = top10_indices.indices + 14950
print("Top 10 most similar entities in KG2:")

for i in range(0,len(top10_global_indices)):
    print(f"Entity ID in DB15K: {top10_global_indices[i]}, Similary Score:{top10_indices.values[i]}")

Top 10 most similar entities in KG2:
Entity ID in DB15K: 15950, Similary Score:0.6474964618682861
Entity ID in DB15K: 15516, Similary Score:0.6449317932128906
Entity ID in DB15K: 16021, Similary Score:0.6376122236251831
Entity ID in DB15K: 15712, Similary Score:0.6364186406135559
Entity ID in DB15K: 15070, Similary Score:0.632177472114563
Entity ID in DB15K: 25124, Similary Score:0.6318706274032593
Entity ID in DB15K: 22237, Similary Score:0.6316174864768982
Entity ID in DB15K: 27700, Similary Score:0.6308556795120239
Entity ID in DB15K: 24284, Similary Score:0.630492091178894
Entity ID in DB15K: 15847, Similary Score:0.6293957829475403


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CVAE(nn.Module):
    def __init__(self, in_dim, hidden_dims, latent_dim=None, entity_type_dim=None, entity_adjust_dim=None, **kwargs):
        super(CVAE, self).__init__()

        if latent_dim is None:
            latent_dim = hidden_dims[-1]

        if entity_type_dim is None:
            entity_type_dim = 0  # Assume 0 if not provided

        self.entity_adjust_dim = entity_adjust_dim
        if self.entity_adjust_dim:
            # Adjust entity_type_dim to the entity_adjust_dim using a linear layer
            self.entity_type_adjuster = nn.Linear(entity_type_dim, entity_adjust_dim)
            adjusted_entity_type_dim = entity_adjust_dim
        else:
            adjusted_entity_type_dim = entity_type_dim

        modules = []

        # Adjust the input dimension of the first encoder layer to include adjusted entity type embeddings
        total_in_dim = in_dim + adjusted_entity_type_dim

        # Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Linear(total_in_dim, h_dim),
                    nn.LeakyReLU(0.1)
                )
            )
            total_in_dim = h_dim  # Update total_in_dim to the output dimension of the current layer

        self.encoder = nn.Sequential(*modules)

        self.fc_mu = nn.Linear(hidden_dims[-1], latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-id], latent_dim)

        # Decoder
        modules = []
        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1])

        hidden_dims.reverse()  # Prepare for symmetric architecture in the decoder

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.Linear(hidden_dims[i], hidden_dims[i + 1]),
                    nn.LeakyReLU(0.1)
                )
            )

        self.decoder = nn.Sequential(*modules)

    def encode(self, x, entity_type):
        if self.entity_adjust_dim:
            # Adjust the dimension of the entity_type embeddings
            entity_type = self.entity_type_adjuster(entity_type)
        
        # Concatenate input x and adjusted entity_type before encoding
        x = torch.cat((x, entity_type), dim=-1)
        x = self.encoder(x)
        mu_x = self.fc_mu(x)
        log_var_x = self.fc_var(x)

        return (mu_x, log_var_x)

    def decode(self, z):
        z = self.decoder_input(z)
        x = self.decoder(z)
        return x



In [53]:
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA

# 假设 Z 是从VAE得到的潜在空间表示
Z = np.random.randn(100, 20)  # 100个样本，20维潜在空间

# 可选：使用PCA降维以减少噪声和过拟合的影响
pca = PCA(n_components=10)
Z_pca = pca.fit_transform(Z)

# 假设 labels 是对应的真实或伪标签
labels = np.random.randint(0, 5, 100)  # 5个簇

# 计算聚类评估指标
silhouette = silhouette_score(Z_pca, labels)
calinski = calinski_harabasz_score(Z_pca, labels)
davies = davies_bouldin_score(Z_pca, labels)

print(f"Silhouette Score: {silhouette}")
print(f"Calinski-Harabasz Score: {calinski}")
print(f"Davies-Bouldin Score: {davies}")


Silhouette Score: -0.04716748430980664
Calinski-Harabasz Score: 0.9561788558325791
Davies-Bouldin Score: 7.862421732009137
